# $D^0 \to K^0_Sπ^+π^-$ with Mixing

We will fit 

$$
D^0 \to K^0_S \pi^+\pi^-
$$

decays generated with the following intermediate resonances:
- $D^0 \to K^0_S \rho(770)$
- $D^0 \to K^{*-}\pi^+$

In addition we add mixing, meaning that $D^0$ oscillates to $\bar{D}^0$, which subsequently decays with a charged-conjugate decay model, depending on time:

$$
|\mathcal{A}(t)|^2 = \frac{1}{4}\left|\mathcal{A}_{D^0} + \frac{q}{p}\mathcal{A}_{\bar{D}^0}\right|^2 \psi_+(t) + 
                     \frac{1}{4}\left|\mathcal{A}_{D^0} - \frac{q}{p}\mathcal{A}_{\bar{D}^0}\right|^2 \psi_-(t) + 
                     2 \Re{\left((\mathcal{A}_{D^0} + \frac{q}{p}\mathcal{A}_{\bar{D}^0})(\mathcal{A}_{D^0} - \frac{q}{p}\mathcal{A}_{\bar{D}^0})^*\right)}\psi_i(t),
$$

with

$$
\begin{align*}
\psi_+(t) &= e^{ -(1-x)\frac{t}{\tau} }\\
\psi_-(t) &= e^{ -(1+x)\frac{t}{\tau} }\\
\psi_i(t) &= e^{ -(1-iy)\frac{t}{\tau} }\\
\end{align*}
$$

Import modules

In [1]:
import os, sys
sys.path.append(os.path.abspath('../'))
print(sys.path)

['/Users/maurizio/Software/AnalysisHelpers/python', '/Users/maurizio/Software/AnalysisHelpers/ganga', '/usr/local/Cellar/root/6.32.02/lib/root', '/usr/local/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/usr/local/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/usr/local/Cellar/python@3.12/3.12.4/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/maurizio/Software/tfa/lib/python3.12/site-packages', '/Users/maurizio/Software/AmpliTF', '/Users/maurizio/Software/AmpliTF/TFA2', '/Users/maurizio/Documents/GitRepositories/tfa_examples']


In [2]:
# Import Tensorflow
import tensorflow as tf

# Import AmpliTF modules
import amplitf.interface as atfi
import amplitf.kinematics as atfk
import amplitf.dynamics as atfd
import amplitf.likelihood as atfl
from amplitf.phasespace.dalitz_phasespace import DalitzPhaseSpace
from develtfa.decaytime_phasespace import DecayTimePhaseSpace
from amplitf.phasespace.combined_phasespace import CombinedPhaseSpace
from develtfa.mixing import Mixing

# Import TFA modules
import tfa.toymc as tft
import tfa.plotting as tfp
import tfa.optimisation as tfo

2024-07-15 23:09:53.279063: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Set the number of events to generate and the number of normalisation point to calculate the integral of the likelihood

In [3]:
ntoys = 10000  # Number of points to generate
nnorm = 100000  # Number of normalisation points

Now define some constants

In [4]:
# Masses of final state particles
from particle.particle import literals as lp
# Dalitz Particles
mkz = atfi.const(lp.K_S_0.mass/1000)
mpi = atfi.const(lp.pi_plus.mass/1000)
md = atfi.const(lp.D_0.mass/1000)

# Resonances
mkst = atfi.const(lp.Kst_892_0.mass/1000)
wkst = atfi.const(lp.Kst_892_0.width/1000)
mrho = atfi.const(lp.rho_770_0.mass/1000)
wrho = atfi.const(lp.rho_770_0.width/1000)

# Blatt-Weisskopf radii for Breit-Wigner lineshape
rd = atfi.const(5.0)
rr = atfi.const(1.5)

The mixing parameters

In [5]:
tdz = atfi.const(1.)
x_mix = atfi.const(0)
y_mix = atfi.const(0)
qop_mix = atfi.const(1)
phi_mix = atfi.const(0)
qoverp = atfi.complex( qop_mix * atfi.cos(phi_mix), 
                       qop_mix * atfi.sin(phi_mix) )

In [6]:
mixing_model = Mixing(tdz,x_mix,y_mix,qop_mix,phi_mix)

and a class to deal with three-body Dalitz phase-space

In [7]:
phsp = DalitzPhaseSpace(mpi, mkz, mpi, md)

Add a class to deal with the decay time

In [8]:
tphsp = DecayTimePhaseSpace(tdz)

And the combined phase-space

In [9]:
c_phsp = CombinedPhaseSpace(phsp,tphsp)

## Model
Below there is a function to define the model, allowing to switch on and off specific components.

In [10]:
def model(x):

    m2ab = phsp.m2ab(x)
    m2bc = phsp.m2bc(x)
    m2ac = phsp.m2ac(x)

    hel_ab = atfd.helicity_amplitude(phsp.cos_helicity_ab(x), 1)
    hel_bc = atfd.helicity_amplitude(phsp.cos_helicity_bc(x), 1)
    hel_ac = atfd.helicity_amplitude(phsp.cos_helicity_ac(x), 1)

    bw1 = atfd.breit_wigner_lineshape(m2ab, mkst, wkst, mpi, mkz, mpi, md, rd, rr, 1, 1)
    bw2 = atfd.breit_wigner_lineshape(m2bc, mkst, wkst, mpi, mkz, mpi, md, rd, rr, 1, 1)
    bw3 = atfd.breit_wigner_lineshape(m2ac, mrho, wrho, mpi, mpi, mkz, md, rd, rr, 1, 1)

    def _model(a1r, a1i, a2r, a2i, a3r, a3i, switches=4 * [1]):

        a1 = atfi.complex(a1r, a1i)
        a2 = atfi.complex(a2r, a2i)
        a3 = atfi.complex(a3r, a3i)

        ampl = atfi.cast_complex(atfi.ones(m2ab)) * atfi.complex(
            atfi.const(0.0), atfi.const(0.0)
        )

        if switches[0]:
            ampl += a1 * bw1 * hel_ab
        if switches[1]:
            ampl += a2 * bw2 * hel_bc
        if switches[2]:
            ampl += a3 * bw3 * hel_ac
        if switches[3]:
            ampl += atfi.cast_complex(atfi.ones(m2ab)) * atfi.complex(
                atfi.const(5.0), atfi.const(0.0)
            )

        return atfd.density(ampl)

    return _model

In [11]:
# Time evolution functions.
@atfi.function
def psip( t, x, tau ):
    r"""Time evolution function $\psi_+(t)$

    Args:
        t (float): decay time of the candidate
        x (float): mixing parameter
        tau (float): lifetime of the decaying particle

    Returns:
        float: the time evolution function for the sum of the two decay amplitudes
    """
    return atfi.exp( - ( 1.0 - x ) * t / tau)

@atfi.function
def psim( t, x, tau ):
    r"""Time evolution function $\psi_-(t)$

    Args:
        t (float): decay time of the candidate
        x (float): mixing parameter
        tau (float): lifetime of the decaying particle

    Returns:
        float: the time evolution function for the sum of the two decay amplitudes
    """
    return atfi.exp( - ( 1.0 + x ) * t / tau)

@atfi.function
def psii( t, y, tau ):
    r"""Time evolution function $\psi_i(t)$

    Args:
        t (float): decay time of the candidate
        y (float): mixing parameter
        tau (float): lifetime of the decaying particle

    Returns:
        float: the time evolution function for the sum of the two decay amplitudes
    """
    return atfi.exp( - atfi.complex( atfi.const(1.0) , -y ) * tf.cast(t / tau, tf.complex128))

In [12]:
def model_mix(x):

    # DZ - MIXING MODEL
    
    # CACHED VARIABLES
    x1 = c_phsp.data1(x)
    x2 = c_phsp.data2(x)
    m2ab = c_phsp.phsp1.m2ab(x1)
    m2bc = c_phsp.phsp1.m2bc(x1)
    m2ac = c_phsp.phsp1.m2ac(x1)

    hel_ab = atfd.helicity_amplitude(c_phsp.phsp1.cos_helicity_ab(x1), 1)
    hel_bc = atfd.helicity_amplitude(c_phsp.phsp1.cos_helicity_bc(x1), 1)
    hel_ac = atfd.helicity_amplitude(c_phsp.phsp1.cos_helicity_ac(x1), 1)

    bw1 = atfd.breit_wigner_lineshape(m2ab, mkst, wkst, mpi, mkz, mpi, md, rd, rr, 1, 1)
    bw2 = atfd.breit_wigner_lineshape(m2bc, mkst, wkst, mpi, mkz, mpi, md, rd, rr, 1, 1)
    bw3 = atfd.breit_wigner_lineshape(m2ac, mrho, wrho, mpi, mpi, mkz, md, rd, rr, 1, 1)

    tep = tf.squeeze(psip(c_phsp.phsp2.t(x2), x_mix, tdz))
    tem = tf.squeeze(psim(c_phsp.phsp2.t(x2), x_mix, tdz))
    tei = tf.squeeze(psii(c_phsp.phsp2.t(x2), y_mix, tdz))

    def _model(a1r, a1i, a2r, a2i, a3r, a3i, switches=4 * [1]):

        a1 = atfi.complex(a1r, a1i)
        a2 = atfi.complex(a2r, a2i)
        a3 = atfi.complex(a3r, a3i)

        # D0 AMPLITUDE
        ampl_dz = atfi.cast_complex(atfi.ones(m2ab)) * atfi.complex(
            atfi.const(0.0), atfi.const(0.0)
        )

        if switches[0]:
            ampl_dz += a1 * bw1 * hel_ab
        if switches[1]:
            ampl_dz += a2 * bw2 * hel_bc
        if switches[2]:
            ampl_dz += a3 * bw3 * hel_ac
        if switches[3]:
            ampl_dz += atfi.cast_complex(atfi.ones(m2ab)) * atfi.complex(
                atfi.const(5.0), atfi.const(0.0)
            )

        # D0bar AMPLITUDE: exchange a <--> c
        ampl_dzb = atfi.cast_complex(atfi.ones(m2ab)) * atfi.complex(
            atfi.const(0.0), atfi.const(0.0)
        )

        if switches[0]:
            ampl_dzb += a1 * bw2 * hel_bc
        if switches[1]:
            ampl_dzb += a2 * bw1 * hel_ab
        if switches[2]:
            ampl_dzb += -a3 * bw3 * hel_ac
        if switches[3]:
            ampl_dzb += atfi.cast_complex(atfi.ones(m2ab)) * atfi.complex(
                atfi.const(5.0), atfi.const(0.0)
            )

        # MIXING
        ampl = atfi.cast_complex(atfi.ones(m2ab)) * atfi.complex(
            atfi.const(0.0), atfi.const(0.0) )

        # return mixing_model.amplitude(c_phsp.phsp2.t(x),ampl_dz,ampl_dzb)
        ampDir = ampl_dz
        ampCnj = qoverp * ampl_dzb

        apb2 = 0.5 * (ampDir + ampCnj)
        amb2 = 0.5 * atfi.conjugate(ampDir - ampCnj)
        dens = atfd.density( apb2 ) * tep
        #print( atfd.density( apb2 ).shape, tep.shape, dens.shape)
        #print( atfd.density( apb2) )
        dens+= atfd.density( amb2 ) * tem
        dens+= 2.0 * atfi.cast_real( apb2 * amb2 * tei )
        #dens+= atfi.density( tf.multiply(apb2 , atfi.complex(tep, atfi.const(0)) ) )
        # mix_model = Mixing(ampl_dz, ampl_dzb, tdz, 
        #                    x_mix, y_mix, qop_mix, phi_mix)
        # ampl = mix_model.amplitude(c_phsp.phsp2.t(x))
        
        return dens

    return _model

**TODO**: The syntax above does not work for the mixing class I though because it multiplies at a certain point a function to a tensor.
I should try to define a full model with mixing as a function and then think how this can be generalised in the code.

## Toy MC Model
The model of the toy MC has all the components on by default. By means of the `switches` flag, some of them can be turned off to see the effects on the fit.

In [13]:
def toymc_model(x, switches=4 * [1]):
    return model(x)(
        switches=switches,
        a1r=atfi.const(1.0),
        a1i=atfi.const(0.0),
        a2r=atfi.const(0.5),
        a2i=atfi.const(0.0),
        a3r=atfi.const(2.0),
        a3i=atfi.const(0.0),
    )

In [14]:
def toymc_model_mix(x, switches=4 * [1]):
    return model_mix(x)(
        switches=switches,
        a1r=atfi.const(1.0),
        a1i=atfi.const(0.0),
        a2r=atfi.const(0.5),
        a2i=atfi.const(0.0),
        a3r=atfi.const(2.0),
        a3i=atfi.const(0.0),
    )

## Likelihood

The Negative Log Likelihood

In [15]:
# TF graph for unbinned negalite log likelihood (the quantity to be minimised)
def nll(data, norm):
    data_model = model_mix(data)
    norm_model = model_mix(norm)

    @atfi.function
    def _nll(pars):
        return atfl.unbinned_nll(data_model(**pars), atfl.integral(norm_model(**pars)))

    return _nll

## Samples

The samples to fit

In [16]:
toy_sample = tft.run_toymc(
    toymc_model_mix, c_phsp, ntoys, maximum=1.0e-20, chunk=1000000, components=False
)

print(toy_sample)

ValueError: in user code:

    File "/Users/maurizio/Software/AmpliTF/TFA2/tfa/toymc.py", line 70, in pdf_vals  *
        d = accept_reject_sample(
    File "/Users/maurizio/Software/AmpliTF/TFA2/tfa/toymc.py", line 30, in accept_reject_sample  *
        return tf.boolean_mask(x, density(x) > r)

    ValueError: Number of mask dimensions must be specified, even if some dimensions are None.  E.g. shape=[None] is ok, but shape=None is not.


and to calculate the integrals

In [47]:
norm_sample = c_phsp.uniform_sample(nnorm)

print(norm_sample)

tf.Tensor(
[[1.43696864 1.63334256 0.32197015]
 [1.55763063 1.05794523 0.87581189]
 [1.22938953 0.83107077 1.28048402]
 ...
 [0.80923705 2.39296498 0.57776134]
 [0.68223338 2.11787583 0.07159538]
 [2.35804056 0.58700197 1.46736376]], shape=(47086, 3), dtype=float64)


Beware that only approximately 1/2 of the events have been generated in the normalisation sample.

## Fit

We are now ready to run the fit. We define first the fit parameters:

In [48]:
pars = [
    tfo.FitParameter("a1r", 1.0, -10.0, 10.0),
    tfo.FitParameter("a1i", 0.0, -10.0, 10.0),
    tfo.FitParameter("a2r", 0.5, -10.0, 10.0),
    tfo.FitParameter("a2i", 0.0, -10.0, 10.0),
    tfo.FitParameter("a3r", 2.0, -10.0, 10.0),
    tfo.FitParameter("a3i", 0.0, -10.0, 10.0),
]

and run Minuit

In [49]:
# Run MINUIT minimisation of the neg. log likelihood
result = tfo.run_minuit(nll(toy_sample, norm_sample), pars)
print(result)
cov = result['covariance']

print(f"{result['time']/result['func_calls']} sec per function call")

fitted_pars = {p: atfi.const(v[0]) for p, v in result["params"].items()}

10 -2084.463946994023 [ 1.0000000e+00 -3.1530751e-07  5.0000000e-01  0.0000000e+00
  2.0000000e+00  0.0000000e+00]
20 -2084.463914880088 [1.         0.         0.5        0.         1.99999906 0.        ]
30 -2092.7227365782846 [ 0.99105059 -0.08457612  0.48731389 -0.00730316  1.98098668 -0.07894469]
40 -2092.8216305720052 [ 0.985217   -0.08476894  0.48515932 -0.01047798  1.97484904 -0.07414026]
50 -2092.8216290813934 [ 0.98521232 -0.08476894  0.48515932 -0.01047795  1.97484904 -0.07414026]
60 -2092.821629083194 [ 0.98521232 -0.08476894  0.48515932 -0.01047798  1.97484904 -0.0741399 ]
70 -2092.821628565892 [ 0.98521232 -0.08476894  0.48515932 -0.01047798  1.97485834 -0.07414026]
80 -2092.821629076737 [ 0.98521232 -0.08476894  0.48515932 -0.01047792  1.97484904 -0.07414026]
90 -2092.821630569737 [ 0.985217   -0.08476894  0.48515932 -0.01047798  1.97484904 -0.0741399 ]
100 -2092.8216285621197 [ 0.98521232 -0.08476894  0.48515932 -0.01047798  1.97485834 -0.0741399 ]
┌─────────────────────

Define a function with the fitted model to calculate fit fractions and projections

In [50]:
def fitted_model(x, switches=4 * [1]):
    return model(x)(**fitted_pars, switches=switches)

The fit fractions are calculated

In [51]:
ff = tfo.calculate_fit_fractions(fitted_model, norm_sample)
print(ff)

[0.2124658758722057, 0.05282594530084564, 0.3925271897073242, 0.43972784820717387]


## Plotting

For plotting the results we generate a sample according to the model and we overlay it to the data

In [52]:
fitted_sample = tft.run_toymc_mix(
    fitted_model, c_phsp, nnorm, maximum=1.0e-20, chunk=1000000, components=True
)

  Updating maximum: 1e-20 -> 967.1933898872821. Starting over.
  Chunk 1, size=27548, total length=27548
  Chunk 2, size=27570, total length=55118
  Chunk 3, size=27410, total length=82528
  Chunk 4, size=27607, total length=110135


In [ ]:
# Plot results
import matplotlib.pyplot as plt

tfp.set_lhcb_style(size=12, usetex=False)  # Adjust plotting style for LHCb papers
fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(10, 6))  # Single subplot on the figure

# Plot 1D histogram from the toy MC sample
tfp.plot_distr2d(
    toy_sample[:, 0],
    toy_sample[:, 1],
    bins=(50, 50),
    ranges=((0.3, 3.1), (0.3, 3.1)),
    fig=fig,
    ax=ax[0, 0],
    labels=(r"$m^2(K_S^0\pi^+)$", r"$m^2(K_S^0\pi^-)$"),
    units=("MeV$^2$", "MeV$^2$"),
    log=True,
)

tfp.plot_distr1d_comparison(
    toy_sample[:, 0],
    fitted_sample[:, 0],
    cweights=[fitted_sample[:, 2 + i] for i in range(4)],
    bins=50,
    range=(0.3, 3.1),
    ax=ax[0, 1],
    label=r"$m^2(K_S^0\pi^+)$",
    units="MeV$^2$",
    legend=False
)

tfp.plot_distr1d_comparison(
    toy_sample[:, 1],
    fitted_sample[:, 1],
    cweights=[fitted_sample[:, 2 + i] for i in range(4)],
    bins=50,
    range=(0.3, 3.1),
    ax=ax[1, 0],
    label=r"$m^2(K_S^0\pi^-)$",
    units="MeV$^2$",
    legend=False
)

tfp.plot_distr1d_comparison(
    phsp.m2ac(toy_sample),
    phsp.m2ac(fitted_sample),
    cweights=[fitted_sample[:, 2 + i] for i in range(4)],
    bins=50,
    range=(0.05, 1.9),
    ax=ax[1, 1],
    label=r"$m^2(\pi^+\pi^-)$",
    units="MeV$^2$",
    legend_ax=ax[1,2]
)

# Show the plot
plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)

In [13]:
tt = c_phsp.filter(c_phsp.unfiltered_sample(1000))

In [54]:
d = tft.accept_reject_sample(
            model_mix, tt
        )

ValueError: Attempt to convert a value (<function model_mix.<locals>._model at 0x12c2cb9c0>) with an unsupported type (<class 'function'>) to a Tensor.

In [42]:
toymc_model_mix(tt)

<tf.Tensor: shape=(452,), dtype=float64, numpy=
array([1.89463407e+01, 3.91388236e+00, 5.01323369e-01, 1.34031183e+01,
       9.10211370e+00, 2.68207225e+01, 2.10930450e+01, 8.87952968e+00,
       4.83019648e+00, 1.28229519e+00, 6.18182753e+00, 5.16585645e+00,
       1.64885356e+01, 1.70303658e+01, 1.77088070e+01, 1.29650929e+01,
       1.10144718e+01, 3.92551141e+00, 3.26166250e-01, 5.00991931e+01,
       5.16208915e+00, 6.67709956e+00, 3.03201028e+00, 2.14039922e+00,
       1.81004987e+01, 1.71487007e+01, 1.66062941e+01, 1.06097180e+02,
       5.67319194e+01, 8.64750180e+00, 2.25409173e+01, 1.53192340e+01,
       1.68287717e+02, 8.17954770e+00, 4.16427538e+01, 1.01148841e+01,
       2.16571606e+02, 9.15007133e+00, 5.78685099e+00, 2.47914842e-01,
       1.05137730e+02, 2.28196190e+02, 1.84726924e+00, 1.18865895e+01,
       7.85298476e+00, 6.61074584e-01, 1.86409782e+01, 2.18434073e-01,
       3.22359515e+01, 3.49763328e+00, 4.83354273e+01, 8.97816719e+00,
       3.90999130e+01, 1.6968

In [28]:
toymc_model(c_phsp.data1(tt))

<tf.Tensor: shape=(452,), dtype=float64, numpy=
array([2.23400958e+01, 4.45414947e+00, 8.39716719e-01, 1.68855265e+01,
       3.15136364e+01, 4.75481816e+01, 3.00806092e+01, 1.10797839e+01,
       5.65884145e+01, 7.90274131e+00, 7.54829634e+00, 3.05407371e+01,
       3.06934074e+01, 4.24854309e+01, 1.74426342e+02, 1.88074393e+01,
       2.95548257e+01, 1.00457142e+01, 1.49102563e+02, 6.41224863e+01,
       5.70001150e+01, 8.81118587e+00, 4.26887895e+00, 8.94830601e+00,
       2.52087442e+01, 4.03286646e+01, 1.74794189e+01, 1.11379357e+02,
       7.75136063e+01, 2.43443924e+01, 3.95918682e+01, 3.32445699e+01,
       2.93931464e+02, 1.43144876e+01, 6.05481514e+01, 5.58801820e+01,
       2.32388297e+02, 1.11238627e+01, 3.05498454e+01, 8.41533001e+00,
       2.05238794e+02, 2.41387105e+02, 2.37776592e+00, 5.48326442e+01,
       3.24491650e+01, 7.36836379e+00, 2.67480562e+01, 1.65942133e+01,
       7.12383773e+01, 5.35121782e+00, 5.51890651e+01, 1.19458755e+01,
       8.20284580e+01, 2.8920

In [39]:
tep = psip(c_phsp.phsp2.t(c_phsp.data2(tt)), x_mix, tdz)
tep = tf.squeeze(tep)
print(tep)

tf.Tensor(
[8.48086813e-01 8.78704763e-01 5.97014872e-01 7.93763716e-01
 2.88830955e-01 5.64074622e-01 7.01217348e-01 8.01417228e-01
 8.53566321e-02 1.62259543e-01 8.18969904e-01 1.69146424e-01
 5.37201211e-01 4.00851903e-01 1.01525990e-01 6.89359814e-01
 3.72679300e-01 3.90764790e-01 2.18752947e-03 7.81304593e-01
 9.05627849e-02 7.57798060e-01 7.10259137e-01 2.39196024e-01
 7.18024607e-01 4.25223618e-01 9.50048409e-01 9.52574908e-01
 7.31896270e-01 3.55215348e-01 5.69331994e-01 4.60804097e-01
 5.72540669e-01 5.71417429e-01 6.87762597e-01 1.81010222e-01
 9.31938523e-01 8.22562412e-01 1.89423250e-01 2.94599073e-02
 5.12270259e-01 9.45353690e-01 7.76892805e-01 2.16779433e-01
 2.42008840e-01 8.97179622e-02 6.96909638e-01 1.31632678e-02
 4.52508223e-01 6.53614448e-01 8.75815295e-01 7.51570461e-01
 4.76662782e-01 5.86721716e-01 3.20541573e-01 7.97742274e-01
 4.14952043e-01 4.45678190e-01 7.57528108e-01 2.29232994e-01
 6.07570372e-01 9.09752424e-01 2.53449060e-01 6.63432944e-01
 5.16298970e-